# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://preview.documentation.bedrock.aws.dev/Documentation/BedrockUserGuide.pdf

## 1. role setting (adding trust relationship)

### 1.1. role check

In [1]:
from sagemaker import get_execution_role

In [2]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20230712T200237


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../images/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../images/inline-policy.jpg)

In [14]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Action': ['bedrock:*'],
   'Resource': '*',
   'Effect': 'Allow',
   'Sid': 'BedrockFullAccess'}]}

### 1.4. Trust relationship
![nn](../images/trust-relationship.jpg)

In [15]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'},
  {'Effect': 'Allow',
   'Principal': {'Service': 'bedrock.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

## 2. Install python SDK for bedrock

In [7]:
!wget https://preview.documentation.bedrock.aws.dev/Documentation/SDK/bedrock-python-sdk.zip
!unzip bedrock-python-sdk.zip -d bedrock-sdk
!rm -rf bedrock-python-sdk.zip

--2023-07-13 02:15:30--  https://preview.documentation.bedrock.aws.dev/Documentation/SDK/bedrock-python-sdk.zip
Resolving preview.documentation.bedrock.aws.dev (preview.documentation.bedrock.aws.dev)... 108.138.85.71, 108.138.85.96, 108.138.85.93, ...
Connecting to preview.documentation.bedrock.aws.dev (preview.documentation.bedrock.aws.dev)|108.138.85.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 249246443 (238M) [application/zip]
Saving to: ‘bedrock-python-sdk.zip’

100%[======================================>] 249,246,443 91.1MB/s   in 2.6s   

2023-07-13 02:15:33 (91.1 MB/s) - ‘bedrock-python-sdk.zip’ saved [249246443/249246443]

Archive:  bedrock-python-sdk.zip
replace bedrock-sdk/.unit-crt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
install_needed = True

In [2]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U ./bedrock-sdk/botocore-1.29.162-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/boto3-1.26.162-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/awscli-1.27.162-py3-none-any.whl
    !{sys.executable} -m pip install -U langchain
    !rm -rf bedrock-sdk

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Processing ./bedrock-sdk/botocore-1.29.162-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/amazon-bedrock/setup/bedrock-sdk/botocore-1.29.162-py3-none-any.whl'

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Processing ./bedrock-sdk/boto3-1.26.162-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/amazon-bedrock/setup/bedrock-sdk/boto3-1.26.162-py3-none-any.whl'

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Processing ./bedrock-sdk/awscli-1.27.162-py3-none-any.wh

## 3. Check setting
아래 메시지가 뜨면 정상 <BR>
![nn](../images/check.png)


In [7]:
import boto3
import langchain

In [8]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.232
boto3 version check: 1.26.162


In [3]:
is_internal_use = True # 내부 직원 용
# is_internal_use = False # 고객 용

In [4]:
if is_internal_use:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name='us-west-2',
        endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )
else:
    bedrock_client = boto3.client(
        service_name='bedrock',
        region_name='us-west-2'
    ) 

In [5]:
output_text = bedrock_client.list_foundation_models()
output_text 

AccessDeniedException: An error occurred (AccessDeniedException) when calling the ListFoundationModels operation: Your account is not authorized to invoke this API operation.